In [3]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, transpile
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.circuit import Parameter, ParameterVector
import numpy as np
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit.extensions import HamiltonianGate
from qiskit.opflow import X, Y, Z, I
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
from functools import partial
from scipy.optimize import minimize as opt
from time import time

def create_hamiltonian(N): #crea l'hamiltoniana generale di ising dati i campi e n di qubits
    for q in range(N): #questo for crea i termini Hz,Hx,Hzz insieme H=gzz*Hz-gx*Hx-gz*Hz
        if q==0:
            opz=Z
            opx=X
            opzz=Z
        else:
            opz=I
            opx=I
            opzz=I
        if q==N-1:
            opzz=Z
        for i in range(1,N):
            if i==q:
                opz=opz^Z
                opx=opx^X
            else:
                opz=opz^I
                opx=opx^I
            if i==q or i==q+1:
                opzz=opzz^Z
            else:
                opzz=opzz^I
        if q==0:
            Hz=opz
            Hx=opx
            Hzz=opzz
        else:
            Hz=Hz+opz
            Hx=Hx+opx
            Hzz=Hzz+opzz
    return Hz,Hx,Hzz

def create_graph(N):
    g=[]
    for i in range(N):
        g.append((i,i+1))
        if i==N-2:
            g.append((N-1,0))
            break
    return g

def create_parameterized_circuit(N,p): #crea il circuito parametrico
    gamma=ParameterVector('gamma',p)
    beta=ParameterVector('beta',p)
    qr=QuantumRegister(N, 'q')
    qc=QuantumCircuit(qr)
    qc.h(qr[0:N])
    g=create_graph(N)
    for i in range(p):
        for edge in g: #cost layer, evolution with gzz*Hzz
            qc.cx(edge[0],edge[1])
            qc.rz(-2*gzz*gamma[i],edge[1])
            qc.cx(edge[0],edge[1])
        qc.rz(-2*gz*gamma[i],qr[0:N]) #cost layer, evolution with -gz*Hz
        qc.rx(-2*gx*beta[i],qr[0:N]) #mixer layer
    return qc

def expectation_value(parameters,estimator,qc,Htot): #calcola il valore dell'energia tramite l'estimator
    qc.assign_parameters(parameters,inplace=True)
    job = estimator.run(qc,Htot)
    result=job.result()
    return result.values[0]

def generate_random_parameters(p): #genera i parametri casualmente tra 0 e pi
    initial_parameters=np.empty(p) #al circuito vanno dati prima i beta in ordine e poi i gamma in ordine
    for i in range(p):
        initial_parameters[i]=np.random.uniform(0,2*np.pi)
    return initial_parameters

def compute_new_parameters(old_parameters,P): #metodo INTERP
    new_parameters=np.empty(P)
    new_parameters[0]=old_parameters[0]
    for i in range(P-1):
        new_parameters[i]=(i/(P-1))*old_parameters[i-1]+((P-1-i)/(P-1))*old_parameters[i]
    new_parameters[P-1]=old_parameters[P-2]
    return new_parameters

def optimization_via_INTERP(N,P_max):
    Hz,Hx,Hzz=create_hamiltonian(N)
    Htot=gzz*Hzz+gx*Hx+gz*Hz
    optimizer=partial(opt,method='L-BFGS-B')
    qc=create_parameterized_circuit(N,1)
    estimator=Estimator()
    gammas=[0.1]
    betas=[0.1]
    new_parameters=betas+gammas
    start_time=time()
    Result=VQE(estimator,qc,optimizer,initial_point=new_parameters).compute_minimum_eigenvalue(Htot)
    optimized_parameters=Result.optimal_point
    step=Result.cost_function_evals
    for P in range(2,P_max+1):
        qc=create_parameterized_circuit(N,P)
        gammas=compute_new_parameters(gammas,P)
        betas=compute_new_parameters(betas,P)
        new_parameters=np.concatenate((betas,gammas))
        Result=VQE(estimator,qc,optimizer,initial_point=new_parameters).compute_minimum_eigenvalue(Htot)
        optimized_parameters=Result.optimal_point
        step=step+Result.cost_function_evals
    comput_time=time()-start_time
    return Result,comput_time,step

def find_the_GS(H):
    matrix_to_diag=H.to_matrix().real
    diag_matrix=np.linalg.eig(matrix_to_diag)
    index=np.argmin(diag_matrix[0])
    return diag_matrix[1][:,index]

def fidelity(H,QAOA_state):
    GS_real=find_the_GS(H)
    fid=np.vdot(QAOA_state,GS_real)
    return (fid.real)**2+(fid.imag)**2

In [59]:
gx=-0.01 #campo trasverso
gz=0 #campo longitudinale
gzz=-1 #coef. davanti all'interazione (-1=ferro,+1=antiferro)
optimizer=partial(opt,method='L-BFGS-B',gtol=10**(-12)) #definisco l'ottimizzatore per VQE

In [123]:
Hz,Hx,Hzz=create_hamiltonian(8)
Htot=gzz*Hzz+gx*Hx+gz*Hz
diag_matrix=np.linalg.eig(Htot.to_matrix().real)
#diag_matrix[0]
diag_matrix[1][:,1]
#f=np.vdot(diag_matrix[1][:,0],state)
#f.real**2+f.imag**2

array([-1.94031258e-03+0.j, -4.85081177e-06+0.j, -4.85081177e-06+0.j,
       -2.42542426e-08+0.j, -4.85081177e-06+0.j, -1.21271913e-08+0.j,
       -2.42542426e-08+0.j, -1.55691327e-10+0.j, -4.85081177e-06+0.j,
       -1.21271121e-08+0.j, -1.21271913e-08+0.j, -6.20037454e-11+0.j,
       -2.42542426e-08+0.j, -6.20037468e-11+0.j, -1.55691360e-10+0.j,
       -5.47933336e-10+0.j, -4.85081177e-06+0.j, -1.21271113e-08+0.j,
       -1.21271121e-08+0.j, -6.16124810e-11+0.j, -1.21271913e-08+0.j,
       -3.08066265e-11+0.j, -6.20037461e-11+0.j, -1.95694295e-10+0.j,
       -2.42542426e-08+0.j, -6.16124795e-11+0.j, -6.20037462e-11+0.j,
       -1.95615079e-10+0.j, -1.55691312e-10+0.j, -1.95694293e-10+0.j,
       -5.47933375e-10+0.j, -7.81242751e-08+0.j, -4.85081177e-06+0.j,
       -1.21271121e-08+0.j, -1.21271113e-08+0.j, -6.16124808e-11+0.j,
       -1.21271121e-08+0.j, -3.07087555e-11+0.j, -6.16124811e-11+0.j,
       -1.56632857e-10+0.j, -1.21271913e-08+0.j, -3.07087469e-11+0.j,
       -3.08066293e-

In [117]:
result=optimization_via_INTERP(6,3)[0]
params=result.optimal_point
qc=create_parameterized_circuit(6,3).assign_parameters(params)
state=Statevector(qc).data
state

array([-2.21416030e-01+6.71532753e-01j, -5.47363995e-04+1.67847532e-03j,
       -5.47363995e-04+1.67847532e-03j, -5.78798561e-06+1.23770441e-05j,
       -5.47363995e-04+1.67847532e-03j, -1.35326047e-06+4.20713385e-06j,
       -5.78798561e-06+1.23770441e-05j, -1.78962312e-07+4.84976736e-06j,
       -5.47363995e-04+1.67847532e-03j, -1.35314004e-06+4.19543966e-06j,
       -1.35326047e-06+4.20713385e-06j, -2.86867388e-08+6.18072162e-08j,
       -5.78798561e-06+1.23770441e-05j, -2.86867388e-08+6.18072163e-08j,
       -1.78962312e-07+4.84976736e-06j, -5.78798561e-06+1.23770441e-05j,
       -5.47363995e-04+1.67847532e-03j, -1.35326047e-06+4.20713385e-06j,
       -1.35314004e-06+4.19543966e-06j, -2.86867388e-08+6.18072162e-08j,
       -1.35326047e-06+4.20713385e-06j, -6.68866926e-09+2.10586511e-08j,
       -2.86867388e-08+6.18072162e-08j, -1.35326047e-06+4.20713385e-06j,
       -5.78798561e-06+1.23770441e-05j, -2.86867388e-08+6.18072162e-08j,
       -2.86867388e-08+6.18072162e-08j, -1.35314004

In [82]:
fidelity(Htot,state)

0.5019403574084427